# Clase 5.4 - API REST Avanzada

**Unidad:** 5 - Arquitectura de Datos Desconectados  

## Objetivos de Aprendizaje

- [ ] Implementar paginacion
- [ ] Crear filtros y busqueda
- [ ] Manejar errores HTTP
- [ ] Versionar API

In [ ]:
from fastapi import FastAPI, Query, HTTPException
from pydantic import BaseModel
from typing import Optional, Generic, TypeVar, List

app = FastAPI()
T = TypeVar('T')

---

## 1. Paginacion

In [ ]:
class Paginado(BaseModel, Generic[T]):
    items: List[T]
    total: int
    pagina: int
    por_pagina: int
    total_paginas: int

@app.get("/tareas")
def listar_tareas(
    pagina: int = Query(1, ge=1),
    por_pagina: int = Query(10, ge=1, le=100)
):
    # Simular datos
    total = 50
    offset = (pagina - 1) * por_pagina
    total_paginas = (total + por_pagina - 1) // por_pagina
    
    return Paginado(
        items=[{"id": i, "titulo": f"Tarea {i}"} for i in range(offset, min(offset + por_pagina, total))],
        total=total,
        pagina=pagina,
        por_pagina=por_pagina,
        total_paginas=total_paginas
    )

---

## 2. Filtros

In [ ]:
@app.get("/tareas")
def listar_tareas_filtradas(
    completada: Optional[bool] = None,
    buscar: Optional[str] = Query(None, min_length=2),
    pagina: int = Query(1, ge=1),
    por_pagina: int = Query(10, ge=1, le=100)
):
    # Aplicar filtros logic
    return {"completada": completada, "buscar": buscar}

---

## 3. Manejo de Errores

In [ ]:
@app.get("/tareas/{tarea_id}")
def obtener_tarea(tarea_id: int):
    if tarea_id > 100:
        raise HTTPException(
            status_code=404,
            detail=f"Tarea {tarea_id} no encontrada"
        )
    return {"id": tarea_id, "titulo": "Tarea ejemplo"}

---

## 4. Versionado

In [ ]:
from fastapi import APIRouter

router_v1 = APIRouter(prefix="/api/v1")
router_v2 = APIRouter(prefix="/api/v2")

@router_v1.get("/tareas")
def listar_v1():
    return {"version": 1}

@router_v2.get("/tareas")
def listar_v2():
    return {"version": 2, "paginacion": True}

app.include_router(router_v1)
app.include_router(router_v2)

---

## Resumen

| Concepto | Implementacion |
|----------|----------------|
| Paginacion | Query params + offset/limit |
| Filtros | Query params opcionales |
| Errores | HTTPException |
| Versionado | APIRouter con prefix |